<a href="https://colab.research.google.com/github/wallematthias/multistack_registration/blob/main/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- Increase heading size -->
<h1 style="font-size: 24px;">Method for Stack Timelapsed Remodelling</h1>

<!-- Author and overview -->
<p><em>Author: Matthias Walle, Institute for Biomechanics, ETH Zurich</em></p>

